In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../data/Heart_Disease_and_Stroke_Mortality_Among_US_Adults_35.csv')

In [3]:
# To see all the columns
pd.set_option('display.max_columns', None)
data.head(3)

,Year,LocationAbbr,LocationDesc,GeographicLevel,DataSource,Class,Topic,Data_Value,Data_Value_Unit,Data_Value_Type,Data_Value_Footnote_Symbol,Data_Value_Footnote,Confidence_limit_Low,Confidence_limit_High,StratificationCategory1,Stratification1,StratificationCategory2,Stratification2,StratificationCategory3,Stratification3,LocationID
0,1999,AL,Autauga,County,NVSS,Cardiovascular Diseases,All heart disease,NaN,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",~,Value suppressed,NaN,NaN,Age group,Ages 35-64 years,Race,American Indian/Alaska Native,Sex,Overall,1001
1,2013,AL,Autauga,County,NVSS,Cardiovascular Diseases,All heart disease,NaN,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",~,Value suppressed,NaN,NaN,Age group,Ages 35-64 years,Race,American Indian/Alaska Native,Sex,Overall,1001
2,2014,AL,Autauga,County,NVSS,Cardiovascular Diseases,All heart disease,NaN,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",~,Value suppressed,NaN,NaN,Age group,Ages 35-64 years,Race,American Indian/Alaska Native,Sex,Overall,1001


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5770240 entries, 0 to 5770239
Data columns (total 21 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Year                        object 
 1   LocationAbbr                object 
 2   LocationDesc                object 
 3   GeographicLevel             object 
 4   DataSource                  object 
 5   Class                       object 
 6   Topic                       object 
 7   Data_Value                  float64
 8   Data_Value_Unit             object 
 9   Data_Value_Type             object 
 10  Data_Value_Footnote_Symbol  object 
 11  Data_Value_Footnote         object 
 12  Confidence_limit_Low        float64
 13  Confidence_limit_High       float64
 14  StratificationCategory1     object 
 15  Stratification1             object 
 16  StratificationCategory2     object 
 17  Stratification2             object 
 18  StratificationCategory3     object 
 19  Stratification3      

## Dropping unwanted columns

In [5]:
data = data.drop(['Class', 'Data_Value_Type','Data_Value_Footnote_Symbol','Data_Value_Footnote','DataSource','StratificationCategory1','StratificationCategory2','StratificationCategory3','GeographicLevel','LocationID'], axis=1)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5770240 entries, 0 to 5770239
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Year                   object 
 1   LocationAbbr           object 
 2   LocationDesc           object 
 3   Topic                  object 
 4   Data_Value             float64
 5   Data_Value_Unit        object 
 6   Confidence_limit_Low   float64
 7   Confidence_limit_High  float64
 8   Stratification1        object 
 9   Stratification2        object 
 10  Stratification3        object 
dtypes: float64(3), object(8)
memory usage: 484.3+ MB


## Renaming Columns

In [7]:
data=data.rename(columns={'Stratification1': 'AgeGroup',
                                'Stratification2': 'RaceEthnicity',
                                'Stratification3': 'Gender',
                                'Data_Value': 'MortalityRate',
                                'Data_Value_Unit': 'Data_Value_Unit',
                                'Confidence_limit_Low' : 'ConfLow',
                                'Confidence_limit_High' : 'ConfHigh',
                                'LocationDesc': 'County',
                                'LocationAbbr': 'State',
                                'Topic': 'CauseOfDeath'})
data.columns

Index(['Year', 'State', 'County', 'CauseOfDeath', 'MortalityRate',
       'Data_Value_Unit', 'ConfLow', 'ConfHigh', 'AgeGroup', 'RaceEthnicity',
       'Gender'],
      dtype='object')

## Checking Null values

- Indentifying null values present in the dataset.
- Checking out for null values in each of the columns.

In [8]:
# Columns with known missing data that are critical for analysis
critical_columns_with_missing_data = [
    'MortalityRate',
    'ConfLow',
    'ConfHigh'
]

# Filter out rows with missing critical data
data = data.dropna(subset=critical_columns_with_missing_data)

# Display the filtered DataFrame to verify
data.head(3)

,Year,State,County,CauseOfDeath,MortalityRate,Data_Value_Unit,ConfLow,ConfHigh,AgeGroup,RaceEthnicity,Gender
53,2016,AL,Autauga,All stroke,25.7,"per 100,000",21.3,30.9,Ages 35-64 years,Overall,Overall
79,2011,AL,Autauga,All stroke,29.5,"per 100,000",22.9,39.7,Ages 35-64 years,Overall,Men
106,2017,AL,Autauga,All stroke,33.6,"per 100,000",25.4,44.1,Ages 35-64 years,Overall,Men


In [9]:
data.isna().sum()

Year               0
State              0
County             0
CauseOfDeath       0
MortalityRate      0
Data_Value_Unit    0
ConfLow            0
ConfHigh           0
AgeGroup           0
RaceEthnicity      0
Gender             0
dtype: int64

In [10]:
# Convert the 'Year' column to numeric, handling errors
data['Year'] = pd.to_numeric(data['Year'], errors='coerce')
data['Year'] = data['Year'].fillna(0)

# Drop any rows where 'Year' could not be converted to a valid integer
data = data.dropna(subset=['Year'])

In [11]:
data['Year'].isna().sum()

np.int64(0)

# Filtering the dataset to include only rows from the most recent years (2014-2019)

In [35]:
data_recent_years = data[data['Year'] >= 2014]

In [36]:
data_recent_years.head()

,Year,State,County,CauseOfDeath,MortalityRate,Data_Value_Unit,ConfLow,ConfHigh,AgeGroup,RaceEthnicity,Gender
53,2016.0,AL,Autauga,All stroke,25.7,"per 100,000",21.3,30.9,Ages 35-64 years,Overall,Overall
106,2017.0,AL,Autauga,All stroke,33.6,"per 100,000",25.4,44.1,Ages 35-64 years,Overall,Men
108,2017.0,AL,Autauga,All heart disease,128.7,"per 100,000",113.1,144.7,Ages 35-64 years,Overall,Overall
109,2016.0,AL,Autauga,All heart disease,128.1,"per 100,000",114.1,145.5,Ages 35-64 years,Overall,Overall
110,2019.0,AL,Autauga,All heart disease,122.6,"per 100,000",106.4,143.4,Ages 35-64 years,Overall,Overall


In [37]:
print(f"The filtered DataFrame (2010-2019) has {len(data_recent_years)} rows.")

The filtered DataFrame (2010-2019) has 888300 rows.


# Checking the unique values.

In [38]:
# Check all columns in the DataFrame for unique values
for column in data_recent_years.columns:
    unique_values = data_recent_years[column].unique()
    if len(unique_values) == 1:
        print(f"The '{column}' column has the same value across all rows.")
        print(f"The single value is: {unique_values[0]}")

The 'Data_Value_Unit' column has the same value across all rows.
The single value is: per 100,000


# Checking for duplicates

In [39]:
# Check for duplicates
duplicate_rows = data_recent_years.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


In [40]:
data_recent_years

,Year,State,County,CauseOfDeath,MortalityRate,Data_Value_Unit,ConfLow,ConfHigh,AgeGroup,RaceEthnicity,Gender
53,2016.0,AL,Autauga,All stroke,25.7,"per 100,000",21.3,30.9,Ages 35-64 years,Overall,Overall
106,2017.0,AL,Autauga,All stroke,33.6,"per 100,000",25.4,44.1,Ages 35-64 years,Overall,Men
108,2017.0,AL,Autauga,All heart disease,128.7,"per 100,000",113.1,144.7,Ages 35-64 years,Overall,Overall
109,2016.0,AL,Autauga,All heart disease,128.1,"per 100,000",114.1,145.5,Ages 35-64 years,Overall,Overall
110,2019.0,AL,Autauga,All heart disease,122.6,"per 100,000",106.4,143.4,Ages 35-64 years,Overall,Overall
...,...,...,...,...,...,...,...,...,...,...,...
5268482,2015.0,WY,Weston,Heart failure,486.4,"per 100,000",415.0,575.4,Ages 65 years and older,White,Overall
5268483,2016.0,WY,Weston,Heart failure,478.3,"per 100,000",397.3,566.1,Ages 65 years and older,White,Overall
5268484,2018.0,WY,Weston,Heart failure,458.8,"per 100,000",393.4,554.6,Ages 65 years and older,White,Overall
5268487,2014.0,WY,Weston,Heart failure,470.0,"per 100,000",404.0,543.2,Ages 65 years and older,White,Overall


In [41]:
# Save the cleaned DataFrame to a new CSV file
data_recent_years.to_csv('../data/cleaned_all_states_Heart_Disease_and_Stroke_Mortality_Among_US_Adults_35.csv', index=False)